In [30]:
import pandas as pd
import os
from datetime import datetime, timedelta
from twarc.client2 import Twarc2
from dotenv import load_dotenv
from pprint import pprint

# Setup environment
load_dotenv()
BEARER_TOKEN = os.getenv('BEARER_TOKEN')
API_KEY = os.getenv('API_KEY')  # cosumer_key
API_KEY_SECRET = os.getenv('API_KEY_SECRET')  # consumer_secret
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')
MY_ID = os.getenv('MY_ID')

# Setup client
t = Twarc2(consumer_key=API_KEY, consumer_secret=API_KEY_SECRET,
           access_token=ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET,
           bearer_token=BEARER_TOKEN, )


def fetch_twitter_data() -> list:
    """
    Uses the twarc library to obtain data of the people I follow.
    """
    users_i_follow = []
    data_generator = t.following(user=MY_ID)

    # Use generator to repeatedly get the next list of users:
    for obj in data_generator:
        users_i_follow.extend(obj['data']) if 'data' in obj.keys() \
            else users_i_follow.extend(obj['errors'])

    return users_i_follow


def get_dataframes() -> tuple:
    """
    This function obtains the data of interest, puts it into
    two pd.DataFrames, and returns a tuple like so:
    (today: pd.DataFrame, yesterday: pd.DataFrame)
    """

    # Initialize dates we will use:
    today = datetime.strftime(datetime.today(), '%b-%-d-%Y')
    yesterday = datetime.strftime((datetime.today() + timedelta(days=-1)), '%b-%-d-%Y')
    two_days_ago = datetime.strftime((datetime.today() + timedelta(days=-2)), '%b-%-d-%Y')

    # Get twitter data and save to csv, this is today's df
    # following_today = fetch_twitter_data()
    # todays_data = pd.json_normalize(following_today)
    # todays_data.to_csv(f'static/{today}.csv')
    todays_data = pd.read_csv(f'static/{today}.csv')

    # Read yesterday's df from file:
    yesterdays_data = pd.read_csv(f'static/{yesterday}.csv')

    # Remove file from two days ago if it exists:
    # FIXME: Ok to uncomment when ready to launch:
    # try:
    #     os.remove(f'static/{two_days_ago}.csv')
    # except FileNotFoundError:
    #     pass

    return todays_data, yesterdays_data



def plant_bad_seed(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function was created as a sanity check. I wanted to
    write a function to plant a difference and see what
    happens to the behavior of the `is_following_list_identical()` function.
    """
    bad_seed = {
        'description': ['some test person'],
        'name': ['john doe'],
        'location': ['somewhere idk'],
        'profile_image_url': ['https://picsum.photos/id/237/200/300'],
        'url': [''],
        'verified': [False],
        'protected': [False],
        'id': ['666'],
        'username': ['jd'],
        'created_at': [datetime.now()],
    }
    bad_df = pd.DataFrame(bad_seed)
    return pd.concat([df, bad_df], ignore_index=True, axis=0)


In [33]:
def is_following_list_identical(todays_df: pd.DataFrame, yesterdays_df: pd.DataFrame) -> tuple:
    # Default data contains a lot of extra columns, so they are dropped:
    extra_columns = ['entities.url.urls', 'entities.description.urls',
                     'public_metrics.followers_count', 'public_metrics.following_count',
                     'public_metrics.tweet_count', 'public_metrics.listed_count',
                     'entities.description.hashtags', 'entities.description.mentions',
                     'pinned_tweet_id', 'entities.description.cashtags',
                     'Unnamed: 0', ]
    todays_df.drop(columns=extra_columns, inplace=True)
    yesterdays_df.drop(columns=extra_columns, inplace=True)

    renamed_columns = ({'id': 'user_id'})
    todays_df.rename(columns=renamed_columns, inplace=True)
    yesterdays_df.rename(columns=renamed_columns, inplace=True)

    # To find differences that exist in either dataframe, we use something called a "symmetric difference"
    # between two sets. We can assume each user_id is unique, so that's what we use for comparison.
    # https://www.adamsmith.haus/python/answers/how-to-get-the-symmetric-difference-of-pandas-dataframes-in-python
    union = pd.concat([todays_df, yesterdays_df], axis=0)
    sym_diff = union.drop_duplicates(subset='user_id', keep=False, inplace=False)

    no_diff = len(sym_diff) == 0  # if empty, there are no differences
    if no_diff:
        return False, sym_diff
    else:
        return True, sym_diff


# td_df = plant_bad_seed(td_df)  # use to test
td_df, yd_df = get_dataframes()
results = is_following_list_identical(todays_df=td_df, yesterdays_df=yd_df)

In [37]:
results[1].head()

,profile_image_url,protected,user_id,name,url,verified,location,created_at,username,description
1130,https://pbs.twimg.com/profile_images/124798892...,False,130284358,Terrance,NaN,False,NaN,2010-04-06T21:42:09.000Z,stickysweett,🦦📸 Published Model. he/him #BLM #BlackTransliv...
14,https://pbs.twimg.com/profile_images/150510449...,False,1436953813743804417,DEZ_🅱️🅾️ND,NaN,False,NaN,2021-09-12T07:24:30.000Z,dez_bond,Country boy living a country life 🦥 🏳️‍🌈 🏴‍☠️
1854,https://pbs.twimg.com/profile_images/732353224...,True,725274207144157184,Flava_flav,NaN,False,NaN,2016-04-27T10:43:40.000Z,legal_feminista,Lawyer. Mother. Aotearoa
